# **HW1: Regression**
In *assignment 1*, you need to finish:

1.  Basic Part: Implement two regression models to predict the Systolic blood pressure (SBP) of a patient. You will need to implement **both Matrix Inversion and Gradient Descent**.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implement one regression model to predict the SBP of multiple patients in a different way than the basic part. You can choose **either** of the two methods for this part.

# **1. Basic Part (55%)**
In the first part, you need to implement the regression to predict SBP from the given DBP


## 1.1 Matrix Inversion Method (25%)


*   Save the prediction result in a csv file **hw1_basic_mi.csv**
*   Print your coefficient


### *Import Packages*

> Note: You **cannot** import any other package

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### *Global attributes*
Define the global attributes

In [2]:
training_dataroot = 'hw1_basic_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_basic_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_basic_mi.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

You can add your own global attributes here


In [3]:
def sigmoid(x):
    return 1 / np.exp(-x)

### *Load the Input File*
First, load the basic input file **hw1_basic_training.csv** and **hw1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [4]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

### *Implement the Regression Model*

> Note: It is recommended to use the functions we defined, you can also define your own functions


#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset
* Validation dataset is used to validate your own model without the testing data



In [5]:
def SplitData():
    '''
    Split training_datalist into training_dataset (~80%) and validation_dataset (~20%)
    '''
    training_list = pd.DataFrame(training_datalist[1:], columns = training_datalist[0]).astype('int')
    training_dataset = training_list[:290].reset_index()
    validation_dataset = training_list[290:].reset_index()
    return training_dataset, validation_dataset


#### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [6]:
def PreprocessData(training_dataset, validation_dataset):
    '''
    Handle outlier by using interquartile range (IQR)
    '''
    Q1_x = np.percentile(training_dataset['dbp'], 25)
    Q3_x = np.percentile(training_dataset['dbp'], 75)
    IQR_x = Q3_x - Q1_x
    lower_limit_x = Q1_x - (1.5 * IQR_x)
    upper_limit_x = Q3_x + (1.5 * IQR_x)
    training_dataset = training_dataset[(training_dataset['dbp'] < upper_limit_x)]
    Q1_y = np.percentile(training_dataset['sbp'], 25)
    Q3_y = np.percentile(training_dataset['sbp'], 75)
    IQR_y = Q3_y - Q1_y
    lower_limit_y = Q1_y - (1.5 * IQR_y)
    upper_limit_y = Q3_y + (1.5 * IQR_y)
    training_dataset = training_dataset[(training_dataset['sbp'] < upper_limit_y)].reset_index()
    x_train, y_train = training_dataset['dbp'], training_dataset['sbp']
    return x_train, y_train


#### Step 3: Implement Regression
> use Matrix Inversion to finish this part




In [7]:
def MatrixInversion(x_train, y_train, max_degrees):
    '''
    Find weight (w) of each feature (phi) by using matrix inversion
    x_train: dbp value of training data
    y_train: sbp value of training data
    max_degrees: maximum degree of polynomial
    '''
    phi = np.array([])
    a = []
    a.append(np.log(x_train[0] + 1))
    a.append(sigmoid(x_train[0]))
    for i in range(max_degrees, -1, -1):
        a.append(np.power(x_train[0], i))
    phi = np.hstack((phi, a))
    for i in range(1, len(x_train)):
        a.clear()
        a.append(np.log(x_train[i] + 1))
        a.append(sigmoid(x_train[i]))
        for j in range(max_degrees, -1, -1):
            a.append(np.power(x_train[i], j))
        phi = np.vstack((phi, a))
    w = np.linalg.inv((np.transpose(phi) @ phi)) @ np.transpose(phi) @ y_train
    return w


#### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*
The final *output_datalist* should look something like this 
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

In [8]:
def MakePrediction(w, max_degrees):
    '''
    Make prediction of testing_datalist
    w: weight of each feature (phi)
    max_degrees: maximum degree of polynomial
    '''
    testing_list = pd.DataFrame(testing_datalist[1:], columns = testing_datalist[0]).astype('int')
    x_test = testing_list['dbp']
    a = 0
    output_datalist = []
    a += w[0] * np.log(x_test[0] + 1)
    a += w[1] * sigmoid(x_test[0])
    for i in range(max_degrees, -1, -1):
        a += w[max_degrees - i + 2] * np.power(x_test[0], i)
    output_datalist = np.hstack((output_datalist, [a]))
    for i in range(1, len(x_test)):
        a = 0
        a += w[0] * np.log(x_test[i] + 1)
        a += w[1] * sigmoid(x_test[i])
        for j in range(max_degrees, -1, -1):
            a += w[max_degrees - j + 2] * np.power(x_test[i], j) 
        output_datalist = np.hstack((output_datalist, [a]))
    return output_datalist


#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```





In [9]:
max_degrees = 1

training_dataset, validation_dataset = SplitData()
x_train, y_train = PreprocessData(training_dataset, validation_dataset)
w = MatrixInversion(x_train, y_train, max_degrees)
output_datalist = []
output = MakePrediction(w, max_degrees)
for i in range(len(output)):
    output_datalist.append([output[i]])
    
print(w)

[-3.70324648e+01 -9.82052221e-49  1.41813979e+00  1.76015008e+02]


### *Write the Output File*
Write the prediction to output csv
> Format: 'sbp'




In [10]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

## 1.2 Gradient Descent Method (30%)


*   Save the prediction result in a csv file **hw1_basic_gd.csv**
*   Output your coefficient update in a csv file **hw1_basic_coefficient.csv**
*   Print your coefficient





### *Global attributes*

In [11]:
output_dataroot = 'hw1_basic_gd.csv' # Output file will be named as 'hw1_basic.csv'
coefficient_output_dataroot = 'hw1_basic_coefficient.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 20 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

coefficient_output = [] # Your coefficient update during gradient descent
                   # Should be a (number of iterations * number_of coefficient) matrix
                   # The format of each row should be ['w0', 'w1', ...., 'wn']

Your own global attributes

### *Load the Input File*
First, load the basic input file **hw1_basic_training.csv** and **hw1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [12]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

### *Implement the Regression Model*


#### Step 1: Split Data

In [13]:
def SplitData():
    '''
    Split training_datalist into training_dataset (~80%) and validation_dataset (~20%)
    '''
    training_list = pd.DataFrame(training_datalist[1:], columns = training_datalist[0]).astype('int')
    training_dataset = training_list[:290].reset_index()
    validation_dataset = training_list[290:].reset_index()
    return training_dataset, validation_dataset

#### Step 2: Preprocess Data

In [14]:
def PreprocessData(training_dataset, validation_dataset):
    '''
    Handle outlier by using interquartile range (IQR)
    '''
    Q1_x = np.percentile(training_dataset['dbp'], 25)
    Q3_x = np.percentile(training_dataset['dbp'], 75)
    IQR_x = Q3_x - Q1_x
    lower_limit_x = Q1_x - (1.5 * IQR_x)
    upper_limit_x = Q3_x + (1.5 * IQR_x)
    training_dataset = training_dataset[(training_dataset['dbp'] < upper_limit_x)]
    Q1_y = np.percentile(training_dataset['sbp'], 25)
    Q3_y = np.percentile(training_dataset['sbp'], 75)
    IQR_y = Q3_y - Q1_y
    lower_limit_y = Q1_y - (1.5 * IQR_y)
    upper_limit_y = Q3_y + (1.5 * IQR_y)
    training_dataset = training_dataset[(training_dataset['sbp'] < upper_limit_y)].reset_index()
    x_train, y_train = training_dataset['dbp'], training_dataset['sbp']
    return x_train, y_train


#### Step 3: Implement Regression
> use Gradient Descent to finish this part

In [15]:
def GradientDescent(x_train, y_train, max_degrees):
    '''
    Find weight (w) of each feature (phi) by using gradient descent
    x_train: dbp value of training data
    y_train: sbp value of training data
    max_degrees: maximum degree of polynomial
    '''
    phi = np.array([])
    a = []
    a.append(np.log(x_train[0] + 1))
    for i in range(max_degrees, -1, -1):
        a.append(np.power(x_train[0], i))
    phi = np.hstack((phi, a))
    for i in range(1, len(x_train)):
        a.clear()
        a.append(np.log(x_train[i] + 1))
        for j in range(max_degrees, -1, -1):
            a.append(np.power(x_train[i], j))
        phi = np.vstack((phi, a))
    w = np.transpose(np.zeros(max_degrees + 2))
    lr = 6e-8
    coefficient_output.append(w)
    for i in range(100000):
        y_predict = phi @ w
        g = -2 * (np.transpose(phi) @ np.transpose(np.array(y_train) - y_predict))
        w = w - lr * g
        coefficient_output.append(w)
    return w

#### Step 4: Make Prediction

Make prediction of testing dataset and store the values in *output_datalist*
The final *output_datalist* should look something like this 
> [ [100], [80], ... , [90] ] where each row contains the predicted SBP

Remember to also store your coefficient update in *coefficient_output*
The final *coefficient_output* should look something like this
> [ [1, 0, 3, 5], ... , [0.1, 0.3, 0.2, 0.5] ] where each row contains the [w0, w1, ..., wn] of your coefficient





In [16]:
def MakePrediction(w, max_degrees):
    '''
    Make prediction of testing_datalist
    w: weight of each feature (phi)
    max_degrees: maximum degree of polynomial
    '''
    testing_list = pd.DataFrame(testing_datalist[1:], columns = testing_datalist[0]).astype('int')
    x_test = testing_list['dbp']
    a = 0
    output_datalist = []
    a += w[0] * np.log(x_test[0] + 1)
    for i in range(max_degrees, -1, -1):
        a += w[max_degrees - i + 1] * np.power(x_test[0], i)
    output_datalist = np.hstack((output_datalist, [a]))
    for i in range(1, len(x_test)):
        a = 0
        a += w[0] * np.log(x_test[i] + 1)
        for j in range(max_degrees, -1, -1):
            a += w[max_degrees - j + 1] * np.power(x_test[i], j) 
        output_datalist = np.hstack((output_datalist, [a]))
    return output_datalist

#### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be:
```
3 2 1
```



In [17]:
max_degrees = 1

training_dataset, validation_dataset = SplitData()
x_train, y_train = PreprocessData(training_dataset, validation_dataset)
w = GradientDescent(x_train, y_train, max_degrees)
output_datalist = []
output = MakePrediction(w, max_degrees)
for i in range(len(output)):
    output_datalist.append([output[i]])

print(w)

[9.36266875 1.02979002 2.91600117]


### *Write the Output File*

Write the prediction to output csv
> Format: 'sbp'

**Write the coefficient update to csv**
> Format: 'w0', 'w1', ..., 'wn'
>*   The number of columns is based on your number of coefficient
>*   The number of row is based on your number of iterations

In [18]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

with open(coefficient_output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in coefficient_output:
    writer.writerow(row)

# **2. Advanced Part (40%)**
In the second part, you need to implement the regression in a different way than the basic part to help your predictions of multiple patients SBP.

You can choose **either** Matrix Inversion or Gradient Descent method.

The training data will be in **hw1_advanced_training.csv** and the testing data will be in **hw1_advanced_testing.csv**.

Output your prediction in **hw1_advanced.csv**

Notice:
> You cannot import any other package other than those given



### Input the training and testing dataset

In [19]:
training_dataroot = 'hw1_advanced_training.csv' # Training data file file named as 'hw1_basic_training.csv'
testing_dataroot = 'hw1_advanced_testing.csv'   # Testing data file named as 'hw1_basic_training.csv'
output_dataroot = 'hw1_advanced.csv' # Output file will be named as 'hw1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be 220 * 1 matrix and saved as numpy array
                      # The format of each row should be ['sbp']

### Your Implementation

### *Load the Input File*

In [20]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = np.array(list(csv.reader(csvfile)))

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = np.array(list(csv.reader(csvfile)))

### *Implement the Regression Model*

#### Step 1: Split Data

In [21]:
def SplitData(id, training_list, shift_time):
    '''
    Split training_datalist into training_dataset (~80%) and validation_dataset (~20%)
    id: subject_id in training_datalist
    training_list: training_datalist
    shift_time: number of shift in sbp column
    '''
    for i in range(1, shift_time + 1):
        training_list[f'sbp[t-{i}]'] = training_list['sbp'].shift(i)
    training_list = training_list.fillna(method='bfill')
    subject_training_dataset = training_list.loc[training_list['subject_id'] == f'{id}'].reset_index()
    train_length = int(len(subject_training_dataset) * 0.8)
    training_dataset = subject_training_dataset[:train_length]
    validation_dataset = subject_training_dataset[train_length:]
    return training_dataset, validation_dataset

#### Step 2: Preprocess Data

In [22]:
def PreprocessData(training_dataset, validation_dataset, shift_time):
    '''
    Handle missing data by deleting and outlier by using interquartile range (IQR)
    '''
    training_dataset = training_dataset[(training_dataset['heartrate'] != '')]
    training_dataset['heartrate'] = training_dataset['heartrate'].astype('int')
    training_dataset['sbp'] = training_dataset['sbp'].astype('int')
    Q1_x = np.percentile(training_dataset['heartrate'], 25)
    Q3_x = np.percentile(training_dataset['heartrate'], 75)
    IQR_x = Q3_x - Q1_x
    lower_limit_x = Q1_x - (1.5 * IQR_x)
    upper_limit_x = Q3_x + (1.5 * IQR_x)
    training_dataset = training_dataset[(training_dataset['heartrate'].astype('int') < upper_limit_x)]
    Q1_y = np.percentile(training_dataset['sbp'], 25)
    Q3_y = np.percentile(training_dataset['sbp'], 75)
    IQR_y = Q3_y - Q1_y
    lower_limit_y = Q1_y - (1.5 * IQR_y)
    upper_limit_y = Q3_y + (1.5 * IQR_y)
    training_dataset = training_dataset[(training_dataset['sbp'] < upper_limit_y)].reset_index()
    x_train = training_dataset['heartrate'].astype('int')
    for i in range(1, shift_time + 1):
        x_train = pd.concat([x_train, training_dataset[f'sbp[t-{i}]']], axis=1)
    x_train = x_train.astype('int')
    y_train = training_dataset['sbp']
    return x_train, y_train

#### Step 3: Implement Regression

In [23]:
def MatrixInversion(x_train, y_train, shift_time, max_degrees):
    '''
    Find weight (w) of each feature (phi) by using matrix inversion
    x_train: heartrate value of training data
    y_train: sbp value of training data
    shift_time: number of shift in sbp column
    max_degrees: maximum degree of polynomial
    '''
    phi = np.array([])
    a = []
    for i in range(1, shift_time + 1):
        a.append(x_train[f'sbp[t-{i}]'][0])
    for i in range(max_degrees, -1, -1):
        a.append(np.power(x_train['heartrate'][0], i))
    phi = np.hstack((phi, a))
    for i in range(1, len(x_train)):
        a.clear()
        for j in range(1, shift_time + 1):
            a.append(x_train[f'sbp[t-{j}]'][i])
        for k in range(max_degrees, -1, -1):
            a.append(np.power(x_train['heartrate'][i], k))
        phi = np.vstack((phi, a))
        a.clear()
    w = np.linalg.inv((np.transpose(phi) @ phi)) @ np.transpose(phi) @ y_train
    return w

#### Step 4: Train Model and Make Prediction

In [24]:
def MakePrediction(dataset, shift_time, max_degrees, w):
    '''
    Make prediction of dataset
    dataset: all data
    shift_time: number of shift in sbp column
    max_degrees: maximum degree of polynomial
    w: weight of each feature (phi)
    '''
    dataset['heartrate'] = dataset['heartrate'].astype('int')
    for i in range(1, shift_time + 1):
        dataset[f'sbp[t-{i}]'] = dataset[f'sbp[t-{i}]'].astype('int')
    dataset['sbp'] = dataset['sbp'].astype('int')
    y_predict = np.array([])

    a = 0
    for i in range(1, shift_time + 1):
        a += w[i - 1] * dataset[f'sbp[t-{i}]'][0]
    for i in range(max_degrees, -1, -1):
        a += w[max_degrees - i + shift_time] * np.power(dataset['heartrate'][0], i)
    y_predict = np.hstack((y_predict, [a]))
    for i in range(1, shift_time + 1):
        dataset[f'sbp[t-{i}]'][i] = a

    for i in range(1, len(dataset['heartrate'])):
        a = 0
        for j in range(1, shift_time + 1):
            a += w[j - 1] * dataset[f'sbp[t-{j}]'][i]
        for k in range(max_degrees, -1, -1):
            a += w[max_degrees - k + shift_time] * np.power(dataset['heartrate'][i], k) 
        y_predict = np.hstack((y_predict, [a]))
        for l in range(1, shift_time + 1):
            dataset[f'sbp[t-{l}]'][i + l] = a
    return y_predict

In [25]:
pd.options.mode.chained_assignment = None 

training_list = pd.DataFrame(training_datalist[1:], columns = training_datalist[0])
testing_list = pd.DataFrame(testing_datalist[1:], columns = testing_datalist[0])
unique_test_subject_id = testing_list['subject_id'].unique().tolist()
shift_time = [2, 7, 1, 9, 8, 2, 8, 3, 8, 5, 3]
max_degrees = [1, 1, 1, 2, 3, 8, 0, 1, 7, 8, 2]

for i in range(len(unique_test_subject_id)):
    subject_id = unique_test_subject_id[i]
    subject_training_dataset = training_list.loc[training_list['subject_id'] == f'{subject_id}'].reset_index()
    subject_testing_dataset = testing_list.loc[testing_list['subject_id'] == f'{subject_id}'].reset_index()
    subject_list = pd.concat([subject_training_dataset, subject_testing_dataset])
    for j in range(1, shift_time[i] + 1):
        subject_list[f'sbp[t-{j}]'] = subject_list['sbp'].shift(j)
    subject_list = subject_list.fillna(method='bfill')
    subject_list = subject_list[(subject_list['heartrate'] != '')].reset_index()
    training_dataset, validation_dataset = SplitData(subject_id, training_list, shift_time[i])
    x_train, y_train = PreprocessData(training_dataset, validation_dataset, shift_time[i])
    w = MatrixInversion(x_train, y_train, shift_time[i], max_degrees[i])
    y_predict = MakePrediction(subject_list, shift_time[i], max_degrees[i], w)
    for j in range(len(subject_testing_dataset)):
        output_datalist.append([y_predict[len(y_predict) - len(subject_testing_dataset) + j]])

### Output your Prediction

> your filename should be **hw1_advanced.csv**

In [26]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered
*   Summarize your work and your reflections
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)